In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [132]:
df = pd.read_csv("dielectron.csv")
df.head()

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
0,147115,366639895,58.71410,-7.31132,10.531000,-57.29740,12.82020,-2.20267,2.17766,1,11.2836,-1.032340,-1.88066,-11.0778,2.14537,-2.344030,-2.072810,-1,8.94841
1,147115,366704169,6.61188,-4.15213,-0.579855,-5.11278,4.19242,-1.02842,-3.00284,-1,17.1492,-11.713500,5.04474,11.4647,12.75360,0.808077,2.734920,1,15.89300
2,147115,367112316,25.54190,-11.48090,2.041680,22.72460,11.66100,1.42048,2.96560,1,15.8203,-1.472800,2.25895,-15.5888,2.69667,-2.455080,2.148570,1,38.38770
3,147115,366952149,65.39590,7.51214,11.887100,63.86620,14.06190,2.21838,1.00721,1,25.1273,4.087860,2.59641,24.6563,4.84272,2.330210,0.565865,-1,3.72862
4,147115,366523212,61.45040,2.95284,-14.622700,-59.61210,14.91790,-2.09375,-1.37154,-1,13.8871,-0.277757,-2.42560,-13.6708,2.44145,-2.423700,-1.684810,-1,2.74718


In [133]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors = 8)


df = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)
df.drop(['Run', 'Event'], axis = 1, inplace = True)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df.loc[:, ~df.columns.str.contains('Q1|Q2')] = scaler.fit_transform(df.loc[:, ~df.columns.str.contains('Q1|Q2')])


In [134]:
X = df.iloc[:, : -1]
y = df.iloc[:, -1]

In [135]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 2)

In [136]:
%%time

from sklearn.neural_network import MLPRegressor
hidden_layers = np.array([20, 10])
mlp = MLPRegressor(hidden_layer_sizes = hidden_layers, activation = 'relu', 
                    solver = 'adam', max_iter = 500, )

mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

CPU times: user 6.89 s, sys: 12.8 ms, total: 6.91 s
Wall time: 6.87 s


0.008476250755006439

In [139]:
%%time

from sklearn.neighbors import KNeighborsRegressor

neigh = KNeighborsRegressor(n_neighbors = 50).fit(X_train, y_train)
y_pred = neigh.predict(X_test)

mean_squared_error(y_test, y_pred)

CPU times: user 7.11 s, sys: 7.92 ms, total: 7.12 s
Wall time: 1.83 s


0.13897597257880495

In [141]:
X_test.shape

(20000, 16)